In [ ]:
import os
import json
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 30

In [ ]:
def load_data(filename):
    with open(filename, "r") as file:
        data = pd.DataFrame(json.load(file))
    data["total_dry_time"]        = data["dry_run_time"]          + data["creation_time"] + data["import_time"]
    data["total_simulation_time"] = data["simulation_times_mean"] + data["creation_time"] + data["import_time"]
    return data

In [ ]:
def plot_bars(precision, nqubits):

    # Load data
    cpu_data = load_data("../library_cpu.dat")
    gpu_data = load_data("../library_gpu.dat")
    gpu_data["library"] += " GPU"
    data = pd.concat([cpu_data, gpu_data])

    
    # Set plot params
    hatches = ['/', '\\', 'o', '-', 'x', '.', '*', '+']
    width = 0.1
    quantities = ["import_time", "creation_time", "dry_run_time",
                  "simulation_times_mean", "total_simulation_time", "total_dry_time"]
    circuit = ["qft", "variational", "supremacy", "qv", "bv"]
    libraries = ["qibo", "qibo GPU", "qiskit", "qiskit-gpu GPU", "qulacs", "qulacs-gpu GPU", "projectq"]
    palette     = sns.color_palette("bright", 7)
    #blues_cp   = sns.color_palette("Blues", 2)
    #oranges_cp = sns.color_palette("Oranges", 2)
    #greens_cp  = sns.color_palette("Greens", 2)
    #reds_cp    = sns.color_palette("Reds", 2)
    ws = [-3*width, -2*width, -width, 0, width, 2 * width, 3*width, 4*width]
    base_condition = (data["nqubits"] == nqubits) & (data["precision"] == precision)

    
    # Plot the results
    xvalues = np.array(range(len(circuit)))
    plt.figure(figsize=(25, 9))
    plt.title(f"{nqubits} qubits - Total dry time")
    for i, library in enumerate(libraries):
        condition = (data["library"] == library) & base_condition
        heights = {q: np.array([float(data[condition & (data["circuit"] == circ)][q]) for circ in circuit])
                   for q in quantities}

        # Set the color and hatch, library by library
        if library == "qibo":
            cp      = palette[0]
            texture = "/"
            label   = "Qibo"
            alpha   = 0.5
        elif library == "qibo GPU":
            cp      = palette[0]
            texture = "/"
            label   = "Qibo GPU"
            alpha   = 1
        elif library == "qiskit":
            cp      = palette[1]
            texture = "-"
            label   = "Qiskit"
            alpha   = 0.5
        elif library == "qiskit-gpu GPU":
            cp      = palette[1]
            texture = "-"
            label   = "Qiskit GPU"
            alpha   = 1
        if library == "qulacs":
            cp      = palette[2]
            texture = "\\"
            label   = "Qulacs"
            alpha   = 0.5
        elif library == "qulacs-gpu GPU":
            cp      = palette[2]
            texture = "\\"
            label   = "Qulacs GPU"
            alpha   = 1
        elif library == "projectq":
            cp      = palette[3]
            texture = "x"
            label   = "ProjectQ"
            alpha   = 0.5

        plt.bar(xvalues + ws[i], heights["total_dry_time"], color=cp,
                align="center", width=width, alpha=alpha, label=label, log=True,
                edgecolor='w', hatch=texture)

    plt.xticks(xvalues, circuit)
    plt.ylabel("Total dry time (sec)")
    plt.legend(fontsize="small", bbox_to_anchor=(1,1))
    plt.savefig(f"libraries_bars_{precision}_{nqubits}.pdf", bbox_inches="tight"),

In [ ]:
plot_bars("double", 30)